In [1]:
import cv2
import mediapipe as mp
import random

mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

# Game constants
WINDOW_WIDTH = 640
WINDOW_HEIGHT = 480
SNAKE_SIZE = 20
FOOD_SIZE = 20

# Initialize the game window
window = cv2.namedWindow("Snake Game", cv2.WINDOW_NORMAL)
cv2.resizeWindow("Snake Game", WINDOW_WIDTH, WINDOW_HEIGHT)

cap = cv2.VideoCapture(0)
hands = mp_hands.Hands()

# Initialize player 1 snake and food positions
snake_x_player1 = [WINDOW_WIDTH // 4]
snake_y_player1 = [WINDOW_HEIGHT // 2]
food_x = random.randint(0, WINDOW_WIDTH - FOOD_SIZE)
food_y = random.randint(0, WINDOW_HEIGHT - FOOD_SIZE)

# Initialize player 2 snake and food positions
snake_x_player2 = [3 * WINDOW_WIDTH // 4]
snake_y_player2 = [WINDOW_HEIGHT // 2]

score_player1 = 0
score_player2 = 0

while True:
    success, image = cap.read()

    image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
    results = hands.process(image)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    if results.multi_hand_landmarks:
        num_hands = len(results.multi_hand_landmarks)
        if num_hands >= 2:
            player1_landmarks = results.multi_hand_landmarks[0]
            player2_landmarks = results.multi_hand_landmarks[1]


            # Player 1 - Get index finger tip coordinates
            index_finger_x_player1 = int(player1_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x * WINDOW_WIDTH)
            index_finger_y_player1 = int(player1_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y * WINDOW_HEIGHT)

            # Player 2 - Get index finger tip coordinates
            index_finger_x_player2 = int(player2_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x * WINDOW_WIDTH)
            index_finger_y_player2 = int(player2_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y * WINDOW_HEIGHT)

            # Player 1 - Move snake based on index finger direction
            print(snake_x_player1)
            if len(snake_x_player1) > 1:
                dx = snake_x_player1[-1] - snake_x_player1[-2]
                dy = snake_y_player1[-1] - snake_y_player1[-2]
                
                # asari ni lekna mildo raixa
                # if abs(dx) > abs(dy):
                #     index_finger_x_player1 += SNAKE_SIZE if dx > 0 else -SNAKE_SIZE
                # else:
                #     index_finger_y_player1 += SNAKE_SIZE if dy > 0 else -SNAKE_SIZE
                
                if abs(dx) > abs(dy):
                # If dx > 0, it means the movement is towards the right
                    if dx > 0:
                        index_finger_x_player1 += SNAKE_SIZE
                    else:
                        index_finger_x_player1 -= SNAKE_SIZE
                else:
                    # If dy > 0, it means the movement is downwards
                    if dy > 0:
                        index_finger_y_player1 += SNAKE_SIZE
                    else:
                        index_finger_y_player1 -= SNAKE_SIZE


                    

            # Player 2 - Move snake based on index finger direction
            if len(snake_x_player2) > 1:
                dx = snake_x_player2[-1] - snake_x_player2[-2]
                dy = snake_y_player2[-1] - snake_y_player2[-2]

                if abs(dx) > abs(dy):
                    index_finger_x_player2 += SNAKE_SIZE if dx > 0 else -SNAKE_SIZE
                else:
                    index_finger_y_player2 += SNAKE_SIZE if dy > 0 else -SNAKE_SIZE

            # Player 1 - Check collision with food
            if abs(snake_x_player1[-1] - food_x) < SNAKE_SIZE and abs(snake_y_player1[-1] - food_y) < SNAKE_SIZE:
                snake_x_player1.append(food_x)
                snake_y_player1.append(food_y)
                food_x = random.randint(0, WINDOW_WIDTH - FOOD_SIZE)
                food_y = random.randint(0, WINDOW_HEIGHT - FOOD_SIZE)
                score_player1 += 1

            # Player 2 - Check collision with food
            if abs(snake_x_player2[-1] - food_x) < SNAKE_SIZE and abs(snake_y_player2[-1] - food_y) < SNAKE_SIZE:
                snake_x_player2.append(food_x)
                snake_y_player2.append(food_y)
                food_x = random.randint(0, WINDOW_WIDTH - FOOD_SIZE)
                food_y = random.randint(0, WINDOW_HEIGHT - FOOD_SIZE)
                score_player2 += 1

            # Player 1 - Move snake
            snake_x_player1.append(index_finger_x_player1)
            snake_y_player1.append(index_finger_y_player1)

            # Player 2 - Move snake
            snake_x_player2.append(index_finger_x_player2)
            snake_y_player2.append(index_finger_y_player2)


            # Comment out the Remove tail segment to make writting place.
            # # Player 1 - Remove tail segment
            if len(snake_x_player1) > 1:
                snake_x_player1.pop(0)
                snake_y_player1.pop(0)

            # Player 2 - Remove tail segment
            if len(snake_x_player2) > 1:
                snake_x_player2.pop(0)
                snake_y_player2.pop(0)

            # Player 1 - Check collision with boundaries
            if (
                snake_x_player1[-1] < 0 or
                snake_x_player1[-1] >= WINDOW_WIDTH or
                snake_y_player1[-1] < 0 or
                snake_y_player1[-1] >= WINDOW_HEIGHT
            ):
                # Game over for player 1
                cv2.putText(image, "Game Over - Player 1", (WINDOW_WIDTH // 2 - 100, WINDOW_HEIGHT // 2), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                snake_x_player1 = [WINDOW_WIDTH // 4]
                snake_y_player1 = [WINDOW_HEIGHT // 2]
                score_player1 = 0

            # Player 2 - Check collision with boundaries
            if (
                snake_x_player2[-1] < 0 or
                snake_x_player2[-1] >= WINDOW_WIDTH or
                snake_y_player2[-1] < 0 or
                snake_y_player2[-1] >= WINDOW_HEIGHT
            ):
                # Game over for player 2
                cv2.putText(image, "Game Over - Player 2", (WINDOW_WIDTH // 2 - 100, WINDOW_HEIGHT // 2), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                snake_x_player2 = [3 * WINDOW_WIDTH // 4]
                snake_y_player2 = [WINDOW_HEIGHT // 2]
                score_player2 = 0

            # Draw player 1 snake
            for x, y in zip(snake_x_player1, snake_y_player1):
                cv2.rectangle(image, (x, y), (x + SNAKE_SIZE, y + SNAKE_SIZE), (0, 255, 0), cv2.FILLED)

            # Draw player 2 snake
            for x, y in zip(snake_x_player2, snake_y_player2):
                cv2.rectangle(image, (x, y), (x + SNAKE_SIZE, y + SNAKE_SIZE), (255, 0, 0), cv2.FILLED)

            # Draw food
            cv2.rectangle(image, (food_x, food_y), (food_x + FOOD_SIZE, food_y + FOOD_SIZE), (0, 0, 255), cv2.FILLED)

    # Draw scores
    cv2.putText(image, f"Player 1 Score: {score_player1}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    cv2.putText(image, f"Player 2 Score: {score_player2}", (WINDOW_WIDTH - 300, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)

    cv2.imshow("Snake Game", image)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


: 